In [1]:
from pathlib import Path

from models.relational.db_schema import SQLite, Table, PostgreSQL, Database
from models.relational.config import DbConfig, DataBaseType
from models.relational.orm.sqlalchemy import SQLAlchemy
from models.relational.metadata import (
    ColumnMeta,
    ColumnType,
    ForeignKeyAction,
    ForeignKeyColumnMeta,
    UniqueColumnsMeta,
)

In [2]:
def get_postgresql_db() -> PostgreSQL[SQLAlchemy, SQLAlchemy.Table]:
    return PostgreSQL[SQLAlchemy, SQLAlchemy.Table](
        db_config=DbConfig(
            host="localhost",
            user="postgres",
            type=DataBaseType.POSTGRESQL,
            password="hugues",
            database="test",
        ),
        orm_class_=SQLAlchemy,
    )

In [3]:
def get_sqlite_db() -> SQLite[SQLAlchemy, SQLAlchemy.Table]:
    db_path = Path.cwd() / "data" / "test.db"
    return SQLite[SQLAlchemy, SQLAlchemy.Table](db_path, SQLAlchemy)

In [4]:
def get_table(
    database: Database[SQLAlchemy, SQLAlchemy.Table],
    table_name: str,
    columns: list[ColumnMeta] | None = None,
    unique_constraints_columns: list[UniqueColumnsMeta] | None = None,
) -> Table[
    SQLAlchemy, SQLAlchemy.Table, SQLAlchemy.Column, SQLAlchemy.ForeignKeyColumn
]:
    """
    Récupérer une table de la base de données
    :param database: Base de données
    :param table_name: Nom de la table
    :param columns: Liste des colonnes de la table
    :param unique_constraints_columns: Liste des contraintes d'unicité
    """

    return Table[
        SQLAlchemy, SQLAlchemy.Table, SQLAlchemy.Column, SQLAlchemy.ForeignKeyColumn
    ](table_name, database, columns, unique_constraints_columns)

In [5]:
database = get_postgresql_db()
# database = get_sqlite_db()

In [6]:
schools_columns: list[ColumnMeta] = [
    ColumnMeta(
        name="Id",
        type=ColumnType.INT,
        length=None,
        nullable=False,
        primary_key=True,
        unique=True,
    ),
    ColumnMeta(
        name="Name",
        type=ColumnType.TEXT,
        length=100,
        nullable=False,
        primary_key=False,
        unique=False,
        default="ENSP Yaounde",
    ),
    ColumnMeta(
        name="Location",
        type=ColumnType.TEXT,
        length=100,
        nullable=False,
        primary_key=False,
        unique=False,
    ),
]

In [7]:
schools = get_table(database, "Schools", schools_columns)

In [8]:
for column in schools.columns:
    print(column)

Colonne Id de type INT de la table Schools
Colonne Name de type VARCHAR de la table Schools
Colonne Location de type VARCHAR de la table Schools


In [9]:
for column in schools._link_table._link_table.columns:
    print(column)

Schools.Id
Schools.Name
Schools.Location


In [10]:
students_columns: list[ColumnMeta] = [
    ColumnMeta(
        name="Id",
        type=ColumnType.INT,
        length=None,
        nullable=False,
        primary_key=True,
        unique=True,
    ),
    ColumnMeta(
        name="Name",
        type=ColumnType.TEXT,
        length=100,
        nullable=False,
        primary_key=False,
        unique=False,
    ),
    ForeignKeyColumnMeta(
        name="SchoolId",
        type=ColumnType.INT,
        length=None,
        nullable=False,
        primary_key=False,
        unique=False,
        foreign_table_name="Schools",
        foreign_column_name="Id",
        on_delete=ForeignKeyAction.CASCADE,
        on_update=ForeignKeyAction.CASCADE,
    ),
]

In [11]:
students = get_table(database, "Students", students_columns)

In [12]:
students_name = students.get_column("Name")

In [13]:
students_name.name = "Fullname"

In [14]:
students._link_table._link_table.columns[1].name

'Fullname'

In [15]:
students_name.name

'Fullname'

In [16]:
students_name._link_column.meta.name

'Fullname'

In [17]:
students_name._link_column._link_column

Column('Fullname', VARCHAR(length=100), table=<Students>, nullable=False)

In [18]:
schools_id = students.get_column("SchoolId")

In [19]:
schools_id.name

'SchoolId'

In [20]:
for column in students.columns:
    print(column)

Colonne Id de type INT de la table Students
Colonne Fullname de type VARCHAR de la table Students
Colonne SchoolId de type INT de la table Students avec clé étrangère vers la table Schools


In [21]:
schema_destination_path = Path.cwd() / "data" / "schema_postgresql.json"
# schema_destination_path = Path.cwd() / "data" / "schema_sqlite.json"

In [22]:
database.get_schema()

{'name': 'test',
 'type': 'postgresql',
 'tables': {'public.Schools': {'name': 'public.Schools',
   'columns': {'Id': {'name': 'Id',
     'type': 'INT',
     'length': None,
     'nullable': False,
     'primary_key': True,
     'unique': False,
     'default': None},
    'Name': {'name': 'Name',
     'type': 'VARCHAR',
     'length': 100,
     'nullable': False,
     'primary_key': False,
     'unique': False,
     'default': "'ENSP Yaounde'::character varying"},
    'Location': {'name': 'Location',
     'type': 'VARCHAR',
     'length': 100,
     'nullable': False,
     'primary_key': False,
     'unique': False,
     'default': None}},
   'unique_columns': {}},
  'public.Students': {'name': 'public.Students',
   'columns': {'Id': {'name': 'Id',
     'type': 'INT',
     'length': None,
     'nullable': False,
     'primary_key': True,
     'unique': False,
     'default': None},
    'Fullname': {'name': 'Fullname',
     'type': 'VARCHAR',
     'length': 100,
     'nullable': False,
 

In [23]:
database.save_schema(schema_destination_path)

In [24]:
new_school_column = ColumnMeta(
    name="Country",
    type=ColumnType.TEXT,
    length=100,
    nullable=False,
    primary_key=False,
    unique=False,
    default="Cameroon",
)

In [25]:
country_column = schools.add_column(new_school_column)

In [26]:
database.save_schema(schema_destination_path)

In [27]:
schools.db_name = "Schools2"

In [28]:
schools.db_name

'Schools2'

In [29]:
schools._link_table.name

'Schools2'

In [30]:
database.save_schema(schema_destination_path)

In [31]:
database.disconnection()